## GeoAI Ground-level NO2 Estimation Challenge by ITU

### Author: Hubert Kłosowski 242424

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Load data

In [ ]:
train = pd.read_csv(os.path.join('data', 'train.csv'))
test = pd.read_csv(os.path.join('data', 'test.csv'))

In [ ]:
train.info()

In [ ]:
test.info()

### Extract date info

In [ ]:
def extract_date_info(dataframe):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format='mixed')
    dataframe['DayOfWeek'] = dataframe['Date'].dt.dayofweek.astype('category')
    dataframe['Month'] = dataframe['Date'].dt.month.astype('category')
    dataframe['Year'] = dataframe['Date'].dt.year.astype('category')
    dataframe['Week'] = dataframe['Date'].dt.isocalendar().week.astype('category')
    dataframe['Season'] = get_season(dataframe['Date']).astype('category')
    dataframe.drop(['Date'], axis=1, inplace=True)
    return dataframe

def get_season(date_series):
    spring = ((date_series.dt.month == 3) & (date_series.dt.day >= 20)) | ((date_series.dt.month > 3) & (date_series.dt.month < 6)) | ((date_series.dt.month == 6) & (date_series.dt.day <= 20))
    summer = ((date_series.dt.month == 6) & (date_series.dt.day >= 21)) | ((date_series.dt.month > 6) & (date_series.dt.month < 9)) | ((date_series.dt.month == 9) & (date_series.dt.day <= 22))
    autumn = ((date_series.dt.month == 9) & (date_series.dt.day >= 23)) | ((date_series.dt.month > 9) & (date_series.dt.month < 12)) | ((date_series.dt.month == 12) & (date_series.dt.day <= 20))
    
    season_series = pd.Series(0, index=date_series.index)  # Domyślnie 0 dla wiosny
    season_series.loc[summer] = 1  # Lato
    season_series.loc[autumn] = 2  # Jesień
    season_series.loc[~(spring | summer | autumn)] = 3  # Zima
    
    return season_series

train = extract_date_info(train)
test = extract_date_info(test)

### Target values in day of week

In [ ]:
sns.barplot(data=train, x='DayOfWeek', y='GT_NO2')

### Target values in months

In [ ]:
sns.barplot(data=train, x='Month', y='GT_NO2')

### Target values in years

In [ ]:
sns.barplot(data=train, x='Year', y='GT_NO2')

### Target values in weeks

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(data=train, x='Week', y='GT_NO2')

### Target values in seasons

In [ ]:
plt.figure(figsize=(15, 10))
sns.barplot(data=train, x='Season', y='GT_NO2')

### Map of id's in train dataset

In [ ]:
import folium


my_map = folium.Map(
    location=(train['LAT'].mean(), train['LON'].mean()),
    zoom_start=7,
)

### Train Locations

In [ ]:
unique_train_locations = train.groupby(['LAT', 'LON'])['GT_NO2'].mean().reset_index()
nans = train.loc[train['GT_NO2'].isna() == True, ['LAT', 'LON']].value_counts().to_frame().reset_index()
unique_train_locations = pd.merge(unique_train_locations, nans, on=['LAT', 'LON'], how='outer')
unique_train_locations.rename({'count': 'Nans'}, axis=1, inplace=True)
unique_train_locations['Nans'] = unique_train_locations['Nans'].apply(lambda x: round(x / 10.96, 2))

In [ ]:
unique_train_locations

In [ ]:
layer_train_map = folium.FeatureGroup(name='Train Locations', show=False)
for index, row in unique_train_locations.iterrows():
    popup_html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <strong>Mean GT_NO2 level:</strong> {row["GT_NO2"]:.2f}<br>
        <strong>Latitude:</strong> {row["LAT"]}<br>
        <strong>Longitude:</strong> {row["LON"]}<br>
        <strong>Percent of nans:</strong> {row['Nans']} %
    </div>
    """
    popup = folium.Popup(popup_html, max_width=300)
    if row['Nans'] > 10:
        folium.Marker(
            location=[row['LAT'], row['LON']],
            icon=folium.Icon(color='green', icon_color='black', icon='home'),
            popup=popup,
        ).add_to(layer_train_map)
    else:
        folium.Marker(
            location=[row['LAT'], row['LON']],
            icon=folium.Icon(color='green', icon='home'),
            popup=popup,
        ).add_to(layer_train_map)
    
layer_train_map.add_to(my_map)

### Test Locations

In [ ]:
unique_test_locations = test[['LAT', 'LON']].drop_duplicates()

layer_test_map = folium.FeatureGroup(name='Test Locations', show=False)
for index, row in unique_test_locations.iterrows():
    popup_html = f"""
    <div style="font-family: Arial; font-size: 14px;">
        <strong>Latitude:</strong> {row["LAT"]}<br>
        <strong>Longitude:</strong> {row["LON"]}
    </div>
    """
    popup = folium.Popup(popup_html, max_width=300)
    folium.Marker(
        location=[row['LAT'], row['LON']],
        icon=folium.Icon(color='red', icon='home'),
        popup=popup,
    ).add_to(layer_test_map)
    
layer_test_map.add_to(my_map)

In [ ]:
folium.LayerControl().add_to(my_map)
my_map.save('my_map.html')

### Prepare data

In [ ]:
test_ids = test['ID_Zindi']
train.reset_index(drop=True, inplace=True)
train.drop(columns=['ID', 'ID_Zindi'], axis=1, inplace=True)
test.drop(columns=['ID', 'ID_Zindi'], axis=1, inplace=True)

### Fill NO2_trop with diff between NO2_total and NO2_strat

In [ ]:
# train_random_num = np.random.uniform(-1e-6, 1e-6, size=len(train))
# test_random_num = np.random.uniform(-1e-6, 1e-6, size=len(test))
# train.fillna({'NO2_trop': train['NO2_total'] - train['NO2_strat'] + train_random_num}, inplace=True)
# test.fillna({'NO2_trop': test['NO2_total'] - test['NO2_strat'] + test_random_num}, inplace=True)

### Get elevation from LAT, LON

In [ ]:
import requests

def get_elevation(lat, lon):
    query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}'
    r = requests.get(query).json()
    return r['results'][0]['elevation']


unique_train_locations['Elevation'] = unique_train_locations.apply(lambda r: get_elevation(r['LAT'], r['LON']), axis=1)
unique_test_locations['Elevation'] = unique_test_locations.apply(lambda r: get_elevation(r['LAT'], r['LON']), axis=1)
unique_train_locations.drop(columns=['Nans', 'GT_NO2'], inplace=True)

train = train.merge(unique_train_locations, on=['LAT', 'LON'], how='left')
test = test.merge(unique_test_locations, on=['LAT', 'LON'], how='left')

### Check ranges of values

In [ ]:
train.describe()

### Make the same ranges for test and train

In [ ]:
def cut_ranges():
    for column in test.select_dtypes(exclude=['category']).columns[2:]:
        test_min = test[column].min()
        test_max = test[column].max()
        to_drop = train[(train[column] < test_min) | (train[column] > test_max)].index
        train.drop(index=to_drop, inplace=True)

cut_ranges()

train.describe()

### Correlation Matrix for train

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(train.corr().round(2), annot=True, cmap='Greys')

### Correlation Matrix for test

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(test.corr().round(2), annot=True, cmap='Greys')

### Scatter plots of columns from original dataset

In [ ]:
def plot_scatter():
    columns = [col for col in train.columns][:10]
    fig, ax = plt.subplots(nrows=len(columns) // 5, ncols=5, figsize=(25, 15))
    for i, col in enumerate(columns):
        x_cord, y_cord = divmod(i, 5)
        sns.scatterplot(data=train, x=col, y='GT_NO2', ax=ax[x_cord, y_cord], s=2)
        ax[x_cord, y_cord].set_title(f'Correlation between {col} and GT_NO2')
        ax[x_cord, y_cord].set_xlabel(col)
        ax[x_cord, y_cord].set_ylabel('GT_NO2')


plot_scatter()

### Do sth with NaNs in GT_NO2

train lightgbm to fill NaNs in train df
filled rows use for final training, or drop

In [ ]:
gt_no2_nans = train.loc[train['GT_NO2'].isna() == True, :]
train.dropna(subset=['GT_NO2'], inplace=True)

### Distribution of values

In [ ]:
train['GT_NO2'].plot(kind='hist')

### Identify outliers in target column using zscore

In [ ]:
from scipy.stats import zscore


detect_outliers = zscore(train['GT_NO2'])

upper_quantiles = pd.DataFrame(list(zip(np.linspace(0.98, 1, 21), [np.quantile(detect_outliers, el) for el in np.linspace(0.98, 1, 21)], [np.quantile(train['GT_NO2'], el) for el in np.linspace(0.98, 1, 21)])), columns=['quantile', 'zscore', 'GT_NO2'])

upper_quantiles

In [ ]:
lower_quantiles = pd.DataFrame(list(zip(np.linspace(0.0, 0.1, 21), [np.quantile(detect_outliers, el) for el in np.linspace(0.0, 0.1, 21)], [np.quantile(train['GT_NO2'], el) for el in np.linspace(0.0, 0.1, 21)])), columns=['quantile', 'zscore', 'GT_NO2'])

lower_quantiles

### Deleting outliers in top 1.4% percentile

In [ ]:
def del_gt_no2_outliers():
    train.reset_index(drop=True, inplace=True)
    indexes_to_drop = []
    q1, q2 = np.quantile(detect_outliers, 0.05), np.quantile(detect_outliers, 0.986)
    for i, el in enumerate(detect_outliers):
        if el > q2:
            indexes_to_drop.append(i)
    train.drop(indexes_to_drop, inplace=True)
    train.reset_index(drop=True, inplace=True)


del_gt_no2_outliers()

train.info()

### Training process

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X, y = train.drop(columns=['GT_NO2'], axis=1), train['GT_NO2']

# scaler = StandardScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=scaler.feature_names_in_)
# test = pd.DataFrame(scaler.transform(test), columns=scaler.feature_names_in_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score, GroupKFold

### LightGBM

In [ ]:
def define_lightgbm_model(trial):
    params = {
        'max_bin': trial.suggest_int('max_bin', 70, 250),
        'num_leaves': trial.suggest_int('num_leaves', 150, 400),
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        'learning_rate': trial.suggest_float('learning_rate', 2e-3, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 400, 700),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.8, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 400),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 1, log=True),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'objective': 'root_mean_squared_error',
        'boosting_type': 'gbdt',
        'tree_learner': 'voting',
        'device': 'cpu',
        'n_jobs': -1,
        'random_state': 4,
        'verbosity': -1,
        # 'categorical_feature': 'name:Week,DayOfWeek,Year,Month,Season'
        'categorical_feature': 'name:Season'
    }
    return lgb.LGBMRegressor(**params)

def objective_lightgbm(trial):
    model = define_lightgbm_model(trial)
    gkf = GroupKFold(n_splits=X['Season'].nunique())
    scores = cross_val_score(model, X, y, groups=X['Season'], cv=gkf, n_jobs=-1, scoring='neg_root_mean_squared_error')
    return scores.mean() * (-1)

### LightGBM study

In [ ]:
study_lightgbm = optuna.create_study(direction='minimize', study_name='GeoAIWithLightGBM', sampler=optuna.samplers.TPESampler())
study_lightgbm.optimize(objective_lightgbm, n_trials=100)

### LightGBM tylko dla najbliższych danych

In [ ]:
# limit_train = pd.read_csv(os.path.join('data', 'train.csv'))
# limit_test = pd.read_csv(os.path.join('data', 'test.csv'))
# 
# limit_train = extract_date_info(limit_train)
# limit_test = extract_date_info(limit_test)
# 
# limit_train.reset_index(drop=True, inplace=True)
# limit_train.drop(columns=['ID', 'ID_Zindi'], axis=1, inplace=True)
# limit_test.drop(columns=['ID', 'ID_Zindi'], axis=1, inplace=True)
# 
# train.dropna(subset=['GT_NO2'], inplace=True)
# 
# limit_X, limit_y = train.drop(columns=['GT_NO2'], axis=1), train['GT_NO2']
# 
# def objective_lightgbm_for_limit_data(trial):
#     model = define_lightgbm_model(trial)
#     gkf = GroupKFold(n_splits=X['DayOfWeek'].nunique())
#     scores = cross_val_score(model, limit_X, limit_y, groups=X['DayOfWeek'], cv=gkf, n_jobs=-1, scoring='neg_root_mean_squared_error')
#     return scores.mean() * (-1)
# 
# study_lightgbm = optuna.create_study(direction='minimize', study_name='GeoAIWithLightGBM', sampler=optuna.samplers.TPESampler())
# study_lightgbm.optimize(objective_lightgbm_for_limit_data, n_trials=200)

### Narazie najlepszy model ever

In [ ]:
# best_params =  {'max_bin': 157, 'num_leaves': 284, 'max_depth': 12, 'learning_rate': 0.018943556979846253, 'n_estimators': 658, 'bagging_fraction': 0.9910723110875617, 'colsample_bytree': 0.4051901668168799, 'min_data_in_leaf': 54, 'reg_lambda': 0.047054010015969996, 'bagging_freq': 3, 'device': 'cpu', 'n_jobs': -1, 'random_state': 4, 'verbosity': -1, 'tree_learner': 'voting', 'objective': 'root_mean_squared_error', 'boosting_type': 'gbdt'}
# 
# lgb_model = lgb.LGBMRegressor(**best_params)
# lgb_model.fit(X, y)

In [ ]:
lgb_model = define_lightgbm_model(study_lightgbm.best_trial)
lgb_model.fit(X, y)

### Use trained model to predict GTNO2 column, 
but in train dataset. Maybe it will learn better train dataset, and in records containing Nans GTNO2 there's sth

In [ ]:
# nans_X = gt_no2_nans.drop(columns=['GT_NO2'], axis=1)
# nans_y = pd.DataFrame.from_dict({'GT_NO2': lgb_model.predict(nans_X)})
# 
# X = pd.concat([nans_X, X], axis=0)
# y = pd.concat([nans_y, y], axis=0)
# 
# lgb_model.fit(X, y)

### Validate model
(just to be here)

In [ ]:
lightgbm_params = ['max_bin', 'num_leaves', 'max_depth', 'learning_rate', 'n_estimators', 'bagging_fraction', 'colsample_bytree', 'min_data_in_leaf']
lgb_pred = lgb_model.predict(X_test)
root_mean_squared_error(y_test, lgb_pred)

In [ ]:
from sklearn.model_selection import LearningCurveDisplay


LearningCurveDisplay.from_estimator(lgb_model, X, y, cv=GroupKFold(n_splits=X['DayOfWeek'].nunique()), groups=X['DayOfWeek'], n_jobs=-1, random_state=4, scoring='neg_root_mean_squared_error')

In [ ]:
# from sklearn.model_selection import LearningCurveDisplay
# 
# 
# LearningCurveDisplay.from_estimator(lgb_model, limit_X, limit_y, cv=GroupKFold(n_splits=X['DayOfWeek'].nunique()), groups=X['DayOfWeek'], n_jobs=-1, random_state=4, scoring='neg_root_mean_squared_error')

### Best params

In [ ]:
study_lightgbm.best_params

In [ ]:
def save_to_csv(y_pred, save_as):
    if 'result' not in os.listdir(os.getcwd()):
        os.mkdir('result')
    final_df = pd.concat([test_ids, pd.DataFrame.from_dict({'GT_NO2': y_pred})], axis=1)
    final_df.to_csv(os.path.join('result', save_as), index=False)
    
save_to_csv(lgb_model.predict(test), 'lightgbm.csv')

In [ ]:
xd = pd.read_csv(os.path.join('result', 'lightgbm.csv'), header=0)
xd['GT_NO2'] *= 0.95
xd.to_csv(os.path.join('result', 'lightgbm_even_better.csv'), index=False)